# Archivos y carpetas desde Python

Cuando abres o guardas un archivo, la ruta se puede especificar como absoluta (es decir, desde la raíz del sistema de archivo en Linux/MacOs o desde el disco en Windows), o relativa desde el directorio de trabajo. Para las rutas relativas se puede usar `..` para indicar el directorio superior.

El directorio de trabajo inicialmente es el directorio desde el que se ha abierto Python. Para ver el directorio de trabajo, usa el comando

In [1]:
import os

print(os.getcwd())

/workspaces/TFM_AlejandroMir/Lecciones


El directorio de trabajo se puede cambiar con este comando:

In [2]:
os.chdir("/workspaces/TFM_AlejandroMir/vscode/")

print(os.getcwd())

os.chdir("..")

print(os.getcwd())

/workspaces/TFM_AlejandroMir/vscode
/workspaces/TFM_AlejandroMir


In [3]:
with open("Lecciones/wilson_01.json") as f:
    print(f.read())

{
  "eft": "SMEFT",
  "basis": "Warsaw",
  "scale": 1000.0,
  "values": {
    "lq1_2223": 1.0000000000000001e-07,
    "lq3_2223": 1.0000000000000001e-07
  }
}


## Localizar los archivos en ejecución

Imagina que queremos abrir el archivo `wilson_01.json` que se encuentra en la misma carpeta que este cuaderno, pero que no sabemos desde qué directorio vamos a ejecutar el archivo. Lo primero que tenemos que hacer es localizar la ruta del cuaderno.

Si estás ejecutando un archivo `.py`, la ruta del archivo está almacenada en la variable `__file__`

In [4]:
%system python /workspaces/TFM_AlejandroMir/Lecciones/aux_02h.py

['/workspaces/TFM_AlejandroMir/Lecciones/aux_02h.py']

y en un cuaderno de Jupyter, para encontrar la ruta al directorio del cuaderno

In [5]:
_dh[0]

PosixPath('/workspaces/TFM_AlejandroMir/Lecciones')

Si abres el cuaderno con la extensión de VSCode, además es posible encontrar la ruta completa al cuaderno:

In [6]:
__vsc_ipynb_file__

'/workspaces/TFM_AlejandroMir/Lecciones/02h.ipynb'

Puedes ver la ruta de cualquier módulo que hayas importado usando `__path__`:

In [7]:
import flavio

print(flavio.__path__)

['/root/.cache/pypoetry/virtualenvs/tfm-alejandromir-_L8EXmOG-py3.10/lib/python3.10/site-packages/flavio']


## Manipular rutas de archivo

Una vez que conocemos la ruta del cuaderno, ahora tenemos que modificarla para obtener la ruta del archivo `wilson_01.json`. Podríamos pensar en modificarla como una cadena de texto, pero esto tiene varios problemas. En primer lugar, en Linux y MacOS se usa como separador de directorios `/`, mientras que en Windows se suele usar `\`. Además, el carácter `\` sirve para representar carácteres especiales, como el salto de línea `\n`, así que para usarlo en una cadena de texto hay que escribir `\\`, o bien usar una "raw-string":

In [8]:
print("C:\\\\Documents")

print(r"C:\\Documents")

C:\\Documents
C:\\Documents


Para evitar estas complicaciones, es mejor usar el módulo `pathlib` de la librería estándar

In [9]:
from pathlib import Path

ruta_cuaderno = Path(__vsc_ipynb_file__)

print(ruta_cuaderno)

/workspaces/TFM_AlejandroMir/Lecciones/02h.ipynb


Para obtener el directorio superior:

In [10]:
print(ruta_cuaderno.parent)

/workspaces/TFM_AlejandroMir/Lecciones


y se pueden añadir los subdirectorios o archivos con el operador `/`, independientemente del símbolo usado por el sistema operativo:

In [11]:
print(ruta_cuaderno.parent / "wilson_01.json")

/workspaces/TFM_AlejandroMir/Lecciones/wilson_01.json


Los objetos `Path` se pueden usar como argumentos de `open()`:

In [18]:
with open(ruta_cuaderno.parent / "wilson_01.json", "rt") as f:
    print(f.read())

{
  "eft": "SMEFT",
  "basis": "Warsaw",
  "scale": 1000.0,
  "values": {
    "lq1_2223": 1.0000000000000001e-07,
    "lq3_2223": 1.0000000000000001e-07
  }
}


Se puede obtener una ruta absoluta a partir de una relativa con el método `absolute()`:

In [12]:
Path(".").absolute()

PosixPath('/workspaces/TFM_AlejandroMir')

Las rutas que contengan `..` o vínculos a otros directorios se pueden simplificar con el método `resolve()`:

In [13]:
(Path(__vsc_ipynb_file__).parent / "../biblio.md").resolve()

PosixPath('/workspaces/TFM_AlejandroMir/biblio.md')

También se puede comprobar si una ruta existe, y si corresponde a un archivo o a un directorio:

In [16]:
print((Path(__vsc_ipynb_file__).parent / "../biblio.md").exists())

print((Path(__vsc_ipynb_file__).parent / "../biblio.md").is_file())

print((Path(__vsc_ipynb_file__).parent / "../biblio.md").is_dir())

True
True
False


Para ver algunos metadatos del archivo o directorio:

In [17]:
(Path(__vsc_ipynb_file__).parent / "../biblio.md").stat()

os.stat_result(st_mode=33188, st_ino=6860251, st_dev=38, st_nlink=1, st_uid=0, st_gid=0, st_size=1840, st_atime=1672935523, st_mtime=1671556947, st_ctime=1671556947)

* `st_size` es el tamaño en bytes.
* `st_atime` es el tiempo del último acceso, en formato de tiempo UNIX.
* `st_mtime` es el tiempo de la última modificación.
* `st_ctime` es el tiempo de la creación en Windows, o de la última modificación de metadatos en Linux.

Para expresar el tiempo en un formato más comprensible:

In [21]:
import datetime

fecha = datetime.datetime.fromtimestamp((Path(__vsc_ipynb_file__).parent / "../biblio.md").stat().st_atime)

print(fecha)

2023-01-05 16:18:43.174986


Para ver los contenidos de un directorio:

In [22]:
for f in Path(__vsc_ipynb_file__).parent.iterdir():
    print(f)

/workspaces/TFM_AlejandroMir/Lecciones/charmloops.png
/workspaces/TFM_AlejandroMir/Lecciones/README.md
/workspaces/TFM_AlejandroMir/Lecciones/.ipynb_checkpoints
/workspaces/TFM_AlejandroMir/Lecciones/01.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/02.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/02c.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/02d.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/02e.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/02f.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/02f.py
/workspaces/TFM_AlejandroMir/Lecciones/input_02c
/workspaces/TFM_AlejandroMir/Lecciones/02g.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/03.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/wilson_01.json
/workspaces/TFM_AlejandroMir/Lecciones/04.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/02b.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/RD_SM.yaml
/workspaces/TFM_AlejandroMir/Lecciones/02h.ipynb
/workspaces/TFM_AlejandroMir/Lecciones/aux_02h.py
